# Précision et efficacité des méthodes 

In [ ]:
#
#    Notebook de cours MAP412 - Chapitre 3 - M. Massot 2020-2021 - Ecole polytechnique
#    ----------   
#    Précision et efficacité des méthodes de quadrature - diagramme coût / précision 
#    
#    Auteurs : L. Séries et M. Massot - (C) 2021
#    


L'erreur de quadrature d'une fonction $f$ sur l'intervalle $[a,b]$, lorsque l'on utilise une formule composée où l'on a découpé l'intervalle en $N$ sous-intervalles de même taille $h=(b -a)/N$ s'écrit :

$$
err =  \int_a^b f(x) {\mathrm d}x - \sum_{j=0}^{N-1} h\,\sum_{i=0}^{s-1}b_i f(x_j+c_i h).
$$

Nous allons évaluer cette erreur en fonction du nombre d'évaluations de la fonction $f$ dans le calcul de la quadrature $fe$ ($fe = N(s-1)+1$ pour une formule de Newton-Cotes). Ce nombre représente une estimation du travail à effectuer et est considéré comme proportionnel au temps calcul sur un ordinateur. 

Pour $N$, dans les expériences qui suivent, nous prenons typiquement $N=1,2,4,8, \ldots 2048 = 2^{11}$ afin d'estimer les résultats pour un seul intervalle, ainsi qu'une séries de découpages uniforme de l'intervalle d'intégration. Nous considérons deux fonctions avec deux dégrés de non-linéarité, chacune dépendant d'une fréquence que l'on prendra valant 2 ou 20 au cours de l'estimation de la précision et de l'efficacité des méthodes. Le but est de représenter des fonctions plus ou moins bien approchées par un polynôme d'interpolation sur l'intervalle d'intégration suivant le degré de la non-linéarité et oscillant plus ou moins afin d'arriver à des conclusions fermes.

Afin d'observer les résultats pour deux fonctions et ces deux fréquences, nous  représentons le logarithme de l'erreur $\log(err)$ en fonction du logarithme du nombre d'évaluations de la fonction $fe$.  Cela  permet de vérifier si l'on a bien une méthode d'ordre donné en observant la décroissance affine du logarithme de l'erreur dans le diagramme logarithmique et d'autre part de comparer les méthodes dans un diagramme coût / précision classique en analyse numérique permettant de comparer l'efficacité des méthodes à précision fixée, ou alors à coût fixé, de comparer la précision ! 


In [ ]:
import numpy as np
import plotly.graph_objs as go
from scipy.integrate import newton_cotes
from scipy.special import roots_legendre

def integral_step(f, xmin, xmax, r, w):
    h = xmax-xmin
    return h*np.sum(w * f(xmin+h*r))

def integrate(f, xmin, xmax, n, stage, quadrature="newton"):
    if quadrature == "newton":
        if (stage==17):
            # poids analytiques sous forme de nombres rationnels
            w = 0.5*np.array([(15043611773/488462349375), (127626606592/488462349375),  (-(3994025216/10854718875)), (166442371072/97692469875), (-(35081792864/8881133625)), (464176543744/54273594375), (-(6806534407936/488462349375)),  (1873775003648/97692469875), (-(75809177572/3618239625)), (1873775003648/97692469875), (-(6806534407936/488462349375)),  (464176543744/54273594375), ( -(35081792864/8881133625)), (166442371072/97692469875), ( -(3994025216/10854718875)), (127626606592/488462349375), (15043611773/488462349375)])
        else:
            w, _ = newton_cotes(stage-1, equal=1)
            w = w/(stage-1)
        r = np.linspace(0, 1, stage)
    elif quadrature == "gauss":
        r, w = roots_legendre(stage)
        r = 0.5*(r+1)
        w = 0.5*w
    else:
        print(f"quadrature : {quadrature} is unknown")
        exit(1)
                
    x = np.linspace(xmin, xmax, n+1)
    res = 0.
    for i in range(n):
        res += integral_step(f, x[i], x[i+1], r, w)
    return res

## Formules de Newton-Cotes 

### Intégration de $f(x) = \cos(\omega x)$

In [ ]:
def f(x, omega):
    return np.cos(omega*x)

In [ ]:
# Intégration sur [0,2]
xmin = 0
xmax = 2

# fréquence
omega = 2

x = np.linspace(xmin,xmax,1000)

fig = go.Figure()
fig.add_trace(go.Scatter(x=x, y=f(omega, x)))
fig.add_shape(type="line", x0=0, y0=0, x1=2, y1=0)
fig.add_shape(type="line", x0=0, y0=-1.1, x1=0, y1=1.1)
buttons=[dict(label="omega=2",  method="update", args=[{"y": [f(x, 2)]}]),
         dict(label="omega=20", method="update", args=[{"y": [f(x, 20)]}])]
fig.update_layout(title={'text':'f(x) = cos(omega x)', 'x':0.5}, updatemenus=[dict(type="buttons", direction="right", buttons=buttons, x = 0.2, y = 1.2)])
fig.show()

In [ ]:
n = np.array([1, 2, 4, 8, 16, 32, 64, 128, 256, 512, 1024, 2048])

err_trap   = np.zeros((2, n.size))
err_newton = np.zeros((2, n.size))
err_boole  = np.zeros((2, n.size))
err_weddle = np.zeros((2, n.size))
err_haut10 = np.zeros((2, n.size))
err_haut17 = np.zeros((2, n.size))

for i, omega_i in enumerate([2, 20]):
    res_exa = (1/omega_i)*np.sin(omega_i*xmax) - (1/omega_i)*np.sin(omega_i*xmin)
    fct = lambda x: f(x, omega_i)

    for j, nj in enumerate(n):
    
        # trapeze
        res_trap = integrate(fct, xmin, xmax, nj, 2, quadrature="newton")
        err_trap[i,j] = abs(res_exa - res_trap)

        # Newton4
        res_newton = integrate(fct, xmin, xmax, nj, 4, quadrature="newton")
        err_newton[i,j] = abs(res_exa - res_newton)
        
        # Boole
        res_boole = integrate(fct, xmin, xmax, nj, 5, quadrature="newton")
        err_boole[i,j] = abs(res_exa - res_boole)

        # Weddle
        res_weddle = integrate(fct, xmin, xmax, nj, 7, quadrature="newton")
        err_weddle[i,j] = abs(res_exa - res_weddle)
        if (err_weddle[i,j]==0):  err_weddle[i,j]=1e-16

        # haut10
        res_haut10 = integrate(fct, xmin, xmax, nj, 10, quadrature="newton")
        err_haut10[i,j] = abs(res_exa - res_haut10)
        if (err_haut10[i,j]==0):  err_haut10[i,j]=1e-16

        #haut17
        res_haut17 = integrate(fct, xmin, xmax, nj, 17, quadrature="newton")
        err_haut17[i,j] = abs(res_exa - res_haut17)
        if (err_haut17[i,j]==0):  err_haut17[i,j]=1e-16

fig = go.Figure()
fig.add_trace(go.Scatter(x=2*n, y=err_trap[0], name="Trapeze (2 etages, ordre 2)"))
fig.add_trace(go.Scatter(x=4*n, y=err_newton[0], name="Newton (4 étages, ordre 4)"))
fig.add_trace(go.Scatter(x=5*n, y=err_boole[0], name="Boole (5 étages, ordre 6)"))
fig.add_trace(go.Scatter(x=7*n, y=err_weddle[0], name="Weddle (7 étages, ordre 8)"))
fig.add_trace(go.Scatter(x=10*n, y=err_haut10[0], name="(10 étages, ordre 11)"))
fig.add_trace(go.Scatter(x=17*n, y=err_haut17[0], name="(17 étages, ordre 18)"))

buttons=[dict(label="omega=2",  method="update", args=[{"y": [err_trap[0], err_newton[0], err_boole[0], err_weddle[0], err_haut10[0], err_haut17[0]]}]),
         dict(label="omega=20", method="update", args=[{"y": [err_trap[1], err_newton[1], err_boole[1], err_weddle[1], err_haut10[1], err_haut17[1]]}])]

fig.update_xaxes(type="log", exponentformat = 'e', title="Nombre d'évaluations de la fonction f(x)")
fig.update_yaxes(type="log", exponentformat = 'e', title="Erreur")

fig.update_layout(updatemenus=[dict(type="buttons", direction="right", buttons=buttons, x = 0.2, y = 1.15)],
                  legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1))

fig.show()

### Intégration de $g(x) = \cos(\omega x) \exp(\sin(\omega x))$

In [ ]:
def g(x, omega):
    return np.cos(omega*x)*np.exp(np.sin(omega*x))

In [ ]:
# Intégration sur [0,2]
xmin = 0
xmax = 2

# fréquence
omega = 2

x = np.linspace(xmin,xmax,1000)

fig = go.Figure()
fig.add_trace(go.Scatter(x=x, y=f(omega, x)))
fig.add_shape(type="line", x0=0, y0=0, x1=2, y1=0)
fig.add_shape(type="line", x0=0, y0=-1.5, x1=0, y1=1.5)
buttons=[dict(label="omega=2",  method="update", args=[{"y": [g(x, 2)]}]),
         dict(label="omega=20", method="update", args=[{"y": [g(x, 20)]}])]
fig.update_layout(title={'text':'g(x) = cos(omega x) exp(sin(omega x))', 'x':0.5}, updatemenus=[dict(type="buttons", direction="right", buttons=buttons, x = 0.2, y = 1.2)])
fig.show()

In [ ]:
n = np.array([1, 2, 4, 8, 16, 32, 64, 128, 256, 512, 1024, 2048])

err_trap   = np.zeros((2, n.size))
err_newton = np.zeros((2, n.size))
err_boole  = np.zeros((2, n.size))
err_weddle = np.zeros((2, n.size))
err_haut10 = np.zeros((2, n.size))
err_haut17 = np.zeros((2, n.size))

for i, omega_i in enumerate([2, 20]):
    res_exa = (1/omega_i)*np.exp(np.sin(omega_i*xmax)) - (1/omega_i)*np.exp(np.sin(omega_i*xmin))
    fct = lambda x: g(x, omega_i)

    for j, nj in enumerate(n):
    
        # trapeze
        res_trap = integrate(fct, xmin, xmax, nj, 2, quadrature="newton")
        err_trap[i,j] = abs(res_exa - res_trap)

        # Newton4
        res_newton = integrate(fct, xmin, xmax, nj, 4, quadrature="newton")
        err_newton[i,j] = abs(res_exa - res_newton)
        
        # Boole
        res_boole = integrate(fct, xmin, xmax, nj, 5, quadrature="newton")
        err_boole[i,j] = abs(res_exa - res_boole)

        # Weddle
        res_weddle = integrate(fct, xmin, xmax, nj, 7, quadrature="newton")
        err_weddle[i,j] = abs(res_exa - res_weddle)
        if (err_weddle[i,j]==0):  err_weddle[i,j]=1e-16

        # haut10
        res_haut10 = integrate(fct, xmin, xmax, nj, 10, quadrature="newton")
        err_haut10[i,j] = abs(res_exa - res_haut10)
        if (err_haut10[i,j]==0):  err_haut10[i,j]=1e-16

        #haut17
        res_haut17 = integrate(fct, xmin, xmax, nj, 17, quadrature="newton")
        err_haut17[i,j] = abs(res_exa - res_haut17)
        if (err_haut17[i,j]==0):  err_haut17[i,j]=1e-16

fig = go.Figure()
fig.add_trace(go.Scatter(x=2*n, y=err_trap[0], name="Trapeze (2 etages, ordre 2)"))
fig.add_trace(go.Scatter(x=4*n, y=err_newton[0], name="Newton (4 étages, ordre 4)"))
fig.add_trace(go.Scatter(x=5*n, y=err_boole[0], name="Boole (5 étages, ordre 6)"))
fig.add_trace(go.Scatter(x=7*n, y=err_weddle[0], name="Weddle (7 étages, ordre 8)"))
fig.add_trace(go.Scatter(x=10*n, y=err_haut10[0], name="(10 étages, ordre 11)"))
fig.add_trace(go.Scatter(x=17*n, y=err_haut17[0], name="(17 étages, ordre 18)"))

buttons=[dict(label="omega=2",  method="update", args=[{"y": [err_trap[0], err_newton[0], err_boole[0], err_weddle[0], err_haut10[0], err_haut17[0]]}]),
         dict(label="omega=20", method="update", args=[{"y": [err_trap[1], err_newton[1], err_boole[1], err_weddle[1], err_haut10[1], err_haut17[1]]}])]

fig.update_xaxes(type="log", exponentformat = 'e', title="Nombre d'évaluations de la fonction g(x)")
fig.update_yaxes(type="log", exponentformat = 'e', title="Erreur")

fig.update_layout(updatemenus=[dict(type="buttons", direction="right", buttons=buttons, x = 0.2, y = 1.15)],
                  legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1))

fig.show()

Plusieurs conclusions peuvent être tirées de ces graphiques : 
- autant dans le cas de la simple fonction cosinus, la première impression est que le plus efficace est de monter en ordre dans la formule de quadrature sur un seul intervalle, autant quand on augmente la fréquence ou que l'on renforce la non-linéarité (voire les deux !), l'utilisation des formules composées s'avère très efficace sans avoir à traiter les questions de conditionnement et stabilité pour un ordre trop important comme on l'a vu précédemment.
- monter en ordre dans le cadre de formules composées est efficace, d'autant plus que l'on cherche une évaluation précise de l'intégrale.
- dans un régimé intermédiaire, plusieurs choix du nombre d'intervalles et de l'ordre permettent d'atteindre la meilleure efficacité. Cela sera à comparer avec les autres types de quadrature (Clenshaw-Curtis et Gauss-Legendre).


## Formules de quadrature de Gauss                 

### Intégration de $f(x) = \cos(\omega x)$  

In [ ]:
n = np.array([1, 2, 4, 8, 16, 32, 64, 128, 256, 512, 1024, 2048])

err_trap   = np.zeros((2, n.size))
err_newton = np.zeros((2, n.size))
err_boole  = np.zeros((2, n.size))
err_weddle = np.zeros((2, n.size))
err_haut17 = np.zeros((2, n.size))
err_gauss_1 = np.zeros((2, n.size))
err_gauss_2 = np.zeros((2, n.size))
err_gauss_3 = np.zeros((2, n.size))
err_gauss_4 = np.zeros((2, n.size))
err_gauss_9 = np.zeros((2, n.size))

for i, omega_i in enumerate([2, 20]):
    res_exa = (1/omega_i)*np.sin(omega_i*xmax) - (1/omega_i)*np.sin(omega_i*xmin)
    fct = lambda x: f(x, omega_i)

    for j, nj in enumerate(n):
        
        # trapeze
        res_trap = integrate(fct, xmin, xmax, nj, 2, quadrature="newton")
        err_trap[i,j] = abs(res_exa - res_trap)
        # Gauss 1
        res_gauss_1 = integrate(fct, xmin, xmax, nj, 1, quadrature="gauss")
        err_gauss_1[i,j] = abs(res_exa - res_gauss_1)

        # Newton4
        res_newton = integrate(fct, xmin, xmax, nj, 4, quadrature="newton")
        err_newton[i,j] = abs(res_exa - res_newton)
        # Gauss 2
        res_gauss_2 = integrate(fct, xmin, xmax, nj, 2, quadrature="gauss")
        err_gauss_2[i,j] = abs(res_exa - res_gauss_2)
        
        # Boole
        res_boole = integrate(fct, xmin, xmax, nj, 5, quadrature="newton")
        err_boole[i,j] = abs(res_exa - res_boole)
        # Gauss 3
        res_gauss_3 = integrate(fct, xmin, xmax, nj, 3, quadrature="gauss")
        err_gauss_3[i,j] = abs(res_exa - res_boole)
        if (err_gauss_3[i,j]==0): err_gauss_3[i,j]=1e-16
            
         # Weddle
        res_weddle = integrate(fct, xmin, xmax, nj, 7, quadrature="newton")
        err_weddle[i,j] = abs(res_exa - res_weddle)
        if (err_weddle[i,j]==0):  err_weddle[i,j]=1e-16
        # Gauss 4
        res_gauss_4 = integrate(fct, xmin, xmax, nj, 4, quadrature="gauss")
        err_gauss_4[i,j] = abs(res_exa - res_weddle)
        if (err_gauss_4[i,j]==0):  err_gauss_4[i,j]=1e-16

        #haut17
        res_haut17 = integrate(fct, xmin, xmax, nj, 17, quadrature="newton")
        err_haut17[i,j] = abs(res_exa - res_haut17)
        if (err_haut17[i,j]==0):  err_haut17[i,j]=1e-16
        # Gauss 9
        res_gauss_9 = integrate(fct, xmin, xmax, nj, 9, quadrature="gauss")
        err_gauss_9[i,j] = abs(res_exa - res_gauss_9)
        if (err_gauss_9[i,j]==0):  err_gauss_9[i,j]=1e-16


fig = go.Figure()
fig.add_trace(go.Scatter(x=n, y=err_gauss_1[0], name="Gauss (1 etage, ordre 2)", line_color="#636EFA"))
fig.add_trace(go.Scatter(x=2*n, y=err_trap[0], name="Trapeze (2 etages, ordre 2)", line_color="#636EFA", line_dash="dash"))

fig.add_trace(go.Scatter(x=2*n, y=err_gauss_2[0], name="Gauss (2 etages, ordre 4)", line_color="#EF553B"))
fig.add_trace(go.Scatter(x=4*n, y=err_newton[0], name="Newton (4 étages, ordre 4)", line_color="#EF553B", line_dash="dash"))

fig.add_trace(go.Scatter(x=3*n, y=err_gauss_3[0], name="Gauss (3 etages, ordre 6)", line_color="#00CC96"))
fig.add_trace(go.Scatter(x=5*n, y=err_boole[0], name="Boole (5 étages, ordre 6)", line_color="#00CC96", line_dash="dash"))

fig.add_trace(go.Scatter(x=4*n, y=err_gauss_4[0], name="Gauss (4 etages, ordre 8)", line_color="#AB63FA"))
fig.add_trace(go.Scatter(x=7*n, y=err_weddle[0], name="Weddle (7 étages, ordre 8)", line_color="#AB63FA", line_dash="dash"))

fig.add_trace(go.Scatter(x=10*n, y=err_gauss_9[0], name="Gauss (9 etages, ordre 18)", line_color="#FFA15A"))
fig.add_trace(go.Scatter(x=17*n, y=err_haut17[0], name="(17 étages, ordre 18)", line_color="#FFA15A", line_dash="dash"))


buttons=[dict(label="omega=2",  method="update", args=[{"y": [err_gauss_1[0], err_trap[0], err_gauss_2[0], err_newton[0],  err_gauss_3[0], err_boole[0], err_gauss_4[0], err_weddle[0], err_gauss_9[0], err_haut17[0]]}]),
         dict(label="omega=20", method="update", args=[{"y": [err_gauss_1[1], err_trap[1], err_gauss_2[1], err_newton[1],  err_gauss_3[1], err_boole[1], err_gauss_4[1], err_weddle[1], err_gauss_9[1], err_haut17[1]]}])]

fig.update_xaxes(type="log", exponentformat = 'e', title="Nombre d'évaluations de la fonction f(x)")
fig.update_yaxes(type="log", exponentformat = 'e', title="Erreur")

fig.update_layout(updatemenus=[dict(type="buttons", direction="right", buttons=buttons, x = 0.2, y = 1.20)],
                  legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1))

fig.show()

### Intégration de $g(x) = \cos(\omega x) \exp(\sin(\omega x)$ 

In [ ]:
n = np.array([1, 2, 4, 8, 16, 32, 64, 128, 256, 512, 1024, 2048])

err_trap   = np.zeros((2, n.size))
err_newton = np.zeros((2, n.size))
err_boole  = np.zeros((2, n.size))
err_weddle = np.zeros((2, n.size))
err_haut17 = np.zeros((2, n.size))
err_gauss_1 = np.zeros((2, n.size))
err_gauss_2 = np.zeros((2, n.size))
err_gauss_3 = np.zeros((2, n.size))
err_gauss_4 = np.zeros((2, n.size))
err_gauss_9 = np.zeros((2, n.size))

for i, omega_i in enumerate([2, 20]):
    res_exa = (1/omega_i)*np.exp(np.sin(omega_i*xmax)) - (1/omega_i)*np.exp(np.sin(omega_i*xmin))
    fct = lambda x: g(x, omega_i)

    for j, nj in enumerate(n):
        
        # trapeze
        res_trap = integrate(fct, xmin, xmax, nj, 2, quadrature="newton")
        err_trap[i,j] = abs(res_exa - res_trap)
        # Gauss 1
        res_gauss_1 = integrate(fct, xmin, xmax, nj, 1, quadrature="gauss")
        err_gauss_1[i,j] = abs(res_exa - res_gauss_1)

        # Newton4
        res_newton = integrate(fct, xmin, xmax, nj, 4, quadrature="newton")
        err_newton[i,j] = abs(res_exa - res_newton)
        # Gauss 2
        res_gauss_2 = integrate(fct, xmin, xmax, nj, 2, quadrature="gauss")
        err_gauss_2[i,j] = abs(res_exa - res_gauss_2)
        
        # Boole
        res_boole = integrate(fct, xmin, xmax, nj, 5, quadrature="newton")
        err_boole[i,j] = abs(res_exa - res_boole)
        # Gauss 3
        res_gauss_3 = integrate(fct, xmin, xmax, nj, 3, quadrature="gauss")
        err_gauss_3[i,j] = abs(res_exa - res_boole)
        if (err_gauss_3[i,j]==0): err_gauss_3[i,j]=1e-16
            
         # Weddle
        res_weddle = integrate(fct, xmin, xmax, nj, 7, quadrature="newton")
        err_weddle[i,j] = abs(res_exa - res_weddle)
        if (err_weddle[i,j]==0):  err_weddle[i,j]=1e-16
        # Gauss 4
        res_gauss_4 = integrate(fct, xmin, xmax, nj, 4, quadrature="gauss")
        err_gauss_4[i,j] = abs(res_exa - res_weddle)
        if (err_gauss_4[i,j]==0):  err_gauss_4[i,j]=1e-16

        #haut17
        res_haut17 = integrate(fct, xmin, xmax, nj, 17, quadrature="newton")
        err_haut17[i,j] = abs(res_exa - res_haut17)
        if (err_haut17[i,j]==0):  err_haut17[i,j]=1e-16
        # Gauss 9
        res_gauss_9 = integrate(fct, xmin, xmax, nj, 9, quadrature="gauss")
        err_gauss_9[i,j] = abs(res_exa - res_gauss_9)
        if (err_gauss_9[i,j]==0):  err_gauss_9[i,j]=1e-16


fig = go.Figure()
fig.add_trace(go.Scatter(x=n, y=err_gauss_1[0], name="Gauss (1 etage, ordre 2)", line_color="#636EFA"))
fig.add_trace(go.Scatter(x=2*n, y=err_trap[0], name="Trapeze (2 etages, ordre 2)", line_color="#636EFA", line_dash="dash"))

fig.add_trace(go.Scatter(x=2*n, y=err_gauss_2[0], name="Gauss (2 etages, ordre 4)", line_color="#EF553B"))
fig.add_trace(go.Scatter(x=4*n, y=err_newton[0], name="Newton (4 étages, ordre 4)", line_color="#EF553B", line_dash="dash"))

fig.add_trace(go.Scatter(x=3*n, y=err_gauss_3[0], name="Gauss (3 etages, ordre 6)", line_color="#00CC96"))
fig.add_trace(go.Scatter(x=5*n, y=err_boole[0], name="Boole (5 étages, ordre 6)", line_color="#00CC96", line_dash="dash"))

fig.add_trace(go.Scatter(x=4*n, y=err_gauss_4[0], name="Gauss (4 etages, ordre 8)", line_color="#AB63FA"))
fig.add_trace(go.Scatter(x=7*n, y=err_weddle[0], name="Weddle (7 étages, ordre 8)", line_color="#AB63FA", line_dash="dash"))

fig.add_trace(go.Scatter(x=10*n, y=err_gauss_9[0], name="Gauss (9 etages, ordre 18)", line_color="#FFA15A"))
fig.add_trace(go.Scatter(x=17*n, y=err_haut17[0], name="(17 étages, ordre 18)", line_color="#FFA15A", line_dash="dash"))


buttons=[dict(label="omega=2",  method="update", args=[{"y": [err_gauss_1[0], err_trap[0], err_gauss_2[0], err_newton[0],  err_gauss_3[0], err_boole[0], err_gauss_4[0], err_weddle[0], err_gauss_9[0], err_haut17[0]]}]),
         dict(label="omega=20", method="update", args=[{"y": [err_gauss_1[1], err_trap[1], err_gauss_2[1], err_newton[1],  err_gauss_3[1], err_boole[1], err_gauss_4[1], err_weddle[1], err_gauss_9[1], err_haut17[1]]}])]

fig.update_xaxes(type="log", exponentformat = 'e', title="Nombre d'évaluations de la fonction g(x)")
fig.update_yaxes(type="log", exponentformat = 'e', title="Erreur")

fig.update_layout(updatemenus=[dict(type="buttons", direction="right", buttons=buttons, x = 0.2, y = 1.20)],
                  legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1))

fig.show()

Plusieurs conclusions peuvent être tirées de ces graphiques : 
- La quadrature de Gauss comparée à une quadrature de Newton-Cotes à ordre fixé est clairement plus efficace.
- monter en ordre dans le cadre de formules composées est très efficace et cela se vérifie aussi dans les régimes de précision intermédiaire et haute. 
- Une stratégie se dégage clairement : une méthode d'ordre élevé de type Gauss-Legendre couplée à une méthode composition avec un nombre raisonnable d'intervalle est la méthode la plus efficace à précision fixée, intermédiaire ou forte. Sous une autre forme, on peut dire qu'à coût fixe dans une gamme raisonnable, on aura l'évaluation la plus précise de l'intégrale avec cette stratégie.
